In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
sys.path.insert(0, "../../code-previous")

In [3]:
import pandas as pd
import numpy as np
import networkx as nx
import sklearn.feature_extraction as skfeatures
import time
import os

import operator

import string
import swifter
import nltk
stopwords = nltk.corpus.stopwords.words('english')


import plotly
import plotly.plotly as py
import plotly.graph_objs as go
plotly.tools.set_credentials_file(username='sreejith_s', api_key='rcet0rTqFCc3WeOzWTjv')



from sklearn.feature_extraction.text import CountVectorizer


from unicodedata import category
from tqdm import tqdm
tqdm.pandas(desc="Progress")

import seaborn as sns
sns.set_style('white')
import matplotlib.pylab as plt

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [4]:
data_path = "/mnt/disks/vault/analysis-data/interdisciplinary-novelty-analysis/all_discipline_data.pql"

#data_path = "/mnt/disks/vault/analysis-data/interdisciplinary-novelty-analysis/physics.pql"
data = pd.read_pickle(data_path)

In [5]:
data.columns

Index(['ArticleID', 'Type', 'Label', 'PubYear', 'Doctypes', 'Title', 'Journal',
       'Journal Abbreviation ISO', 'title_without_stopwords'],
      dtype='object')

In [6]:
physics_labels = ['Physics, Applied',
                  'Physics, Fluids & Plasmas',
                  'Physics, Atomic, Molecular & Chemical',
                  'Physics, Multidisciplinary',
                  'Physics, Condensed Matter',
                  'Physics, Nuclear',
                  'Physics, Particles & Fields',
                  'Physics, Mathematical']

In [7]:
physics_data = data[data["Label"].isin(physics_labels)]
article_df = None

In [8]:
row_counts = physics_data.groupby('PubYear').size().reset_index(name="count")
counts =  list(row_counts["count"])

#physics_applied_phy = physics_data[physics_data.Label == "Physics, Nuclear"]
physics_data = physics_data.sort_values(by="PubYear")
vectorizer_physics = CountVectorizer()
vectorized_words_physics = vectorizer_physics.fit_transform(physics_data.title_without_stopwords)

In [9]:
N = 200

vocab = vectorizer_physics.vocabulary_
rev_vocab = {index:word for word,index in vocab.items()}
# Highest Frequency words all time
all_time_word_frequencies = vectorized_words_physics.sum(axis=0).reshape(-1,).tolist()[0]
word_index_with_freq = {index:freq for index,freq in enumerate(all_time_word_frequencies)}

word_with_freq = {rev_vocab[index]:np.log(freq + 1) for index, freq in word_index_with_freq.items()}

sorted_word_with_freq = sorted(word_with_freq.items(), key=operator.itemgetter(1), reverse=True)

In [10]:
index_used_only_once = [index for (index, freq) in word_index_with_freq.items() if freq == 1]

In [26]:
_word_matrix_of_singletime_words = vectorized_words_physics[:,index_used_only_once]

__Find horizontal sum__

In [27]:
horizontal_sums = _word_matrix_of_singletime_words.sum(axis=1).reshape(-1,)

In [35]:
hsums = horizontal_sums.reshape(-1).tolist()[0]
title_indices = [index for index,value in enumerate(hsums) if value >= 1]

108952

In [36]:
physics_titles_with_singletime_used_word = physics_data.iloc[title_indices]

In [37]:
len(physics_titles_with_singletime_used_word)

108952

  6%|▋         | 258509/4056377 [2:59:53<44:34:37, 23.67it/s] 

KeyboardInterrupt: 